In [36]:
import nibabel as nib
import numpy as np
from nilearn import datasets,maskers, plotting,image,connectome
from bids.layout import BIDSLayout

In [37]:
#base directory
fmriprep_dir='../Full-Dataset'
layout=BIDSLayout(fmriprep_dir)

In [38]:
len(layout.get_subjects())

71

In [39]:
# Fetch the AAL atlas
aal_atlas = datasets.fetch_atlas_aal('SPM12')

# Load the AAL atlas image
aal_img = nib.load(aal_atlas.maps)

In [58]:
fmri_files=layout.get(subject='01',datatype='func',extension='.nii.gz',return_type="file")
# Assuming you have an fMRI image 'fmri_img', load it using nibabel
fmri_img = nib.load(r'C:\Users\reub\Downloads\filtered_func_data.nii.gz')

# Extract data from the fMRI image
fmri_data = fmri_img.get_fdata()

In [59]:
# Resample AAL atlas to match the fMRI image dimensions
aal_img_resampled = image.resample_to_img(aal_img, fmri_img, interpolation='nearest')

In [60]:
aal_img_resampled.shape

(96, 96, 32)

In [61]:
# Create an array to store the ROI data
roi_data = np.zeros((len(aal_atlas.labels), fmri_data.shape[-1]))

In [65]:
roi_mask = (aal_img_resampled.get_fdata() == 2001)
roi_mask.shape

(96, 96, 32)

In [69]:
# Iterate through each region in the AAL atlas
for i, label in enumerate(aal_atlas.labels):
    # Get the region ID corresponding to the label
    region_id = int(aal_atlas.indices[i])

    # Mask the fMRI data with the region's binary mask from the atlas
    roi_mask = (aal_img_resampled.get_fdata() == region_id)
    roi_data[i, :] = np.mean(fmri_data[roi_mask], axis=0)

# Now, 'roi_data' contains the mean time series for each ROI

# Optionally, you can also obtain the coordinates of each ROI
roi_coordinates = np.array([np.where(aal_img_resampled.get_fdata() == int(aal_atlas.indices[i])) for i in range(len(aal_atlas.labels))])

C:\Users\reub\AppData\Local\Temp\ipykernel_12940\2700845486.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  roi_coordinates = np.array([np.where(aal_img_resampled.get_fdata() == int(aal_atlas.indices[i])) for i in range(len(aal_atlas.labels))])


In [70]:
roi_data.shape

(116, 336)